In [151]:
import Unicode: isletter
using BenchmarkTools

In [153]:
function isletter(s::String)::Bool
    all(isletter.(c for c in s))
end

function read_input(filename)
    rows = open(filename) do f
        readlines(f)
    end
    n_rows = length(rows)
    n_cols = length(rows[1])
    d = Dict{Tuple{Int64, Int64}, String}()
    for i = 3:n_rows-2
        for j = 3:n_cols-2
            c = string(rows[i][j])
            if c == "."
                if isletter(rows[i-1][j])
                    c = rows[i-2][j] * rows[i-1][j]
                elseif isletter(rows[i+1][j])
                    c = rows[i+1][j] * rows[i+2][j]
                elseif isletter(rows[i][j-1])
                    c = rows[i][j-2] * rows[i][j-1]
                elseif isletter(rows[i][j+1])
                    c = rows[i][j+1] * rows[i][j+2]
                end
            elseif c == " " || isletter(c)
                c = "#"
            end
            d[(i-2, j-2)] = c
        end
    end
    d, n_rows - 4, n_cols - 4   
end

function teleport(maze, p)
    port = maze[p]
    for (key, val) in maze
        if val == port && key != p
            return key
        end
    end
end

function get_adjacent(maze, n_rows, n_cols, p)
    i, j = p
    adj = []
    if isletter(maze[p]) && maze[p] != "AA"
        push!(adj, teleport(maze, p))
    end
    for q in ((i-1, j), (i+1, j), (i, j-1), (i, j+1))
        if !((1 <= q[1] <= n_rows) && (1 <= q[2] <= n_cols))
            continue
        elseif maze[q] == "." || isletter(maze[q])
            push!(adj, q)
        end
    end
    adj
end

function find_start_goal(maze)
    start = (0, 0)
    goal = (0, 0)
    for (key, val) in maze
        if val == "AA"
            start = key
        elseif val == "ZZ"
            goal = key
        end
    end
    start, goal
end

function bfs(maze, n_rows, n_cols, p_start)
    queue = []
    explored = Set()
    prev = Dict()
    push!(explored, p_start)
    push!(queue, p_start)
    while length(queue) > 0
        v = popfirst!(queue)
        if maze[v] == "ZZ"
            return prev
        end
        for w in get_adjacent(maze, n_rows, n_cols, v)
            if w ∉ explored
                push!(explored, w)
                push!(queue, w)
                prev[w] = v
            end
        end
    end
end

function find_path(prev, p_start, p_goal)
    path = Tuple{Int64, Int64}[]
    push!(path, p_goal)
    p = p_goal
    while prev[p] != p_start
        push!(path, prev[p])
        p = prev[p]
    end
    push!(path, p_start)
    reverse(path), length(path) - 1
end

function runit(filename)
    maze, n_rows, n_cols = read_input(filename)
    p_start, p_goal = find_start_goal(maze)
    prev = bfs(maze, n_rows, n_cols, p_start)
    path, dist = find_path(prev, p_start, p_goal)
    dist
end

runit (generic function with 1 method)

In [154]:
runit("20_input.txt")

552

In [155]:
@btime runit("20_input.txt")

  7.136 ms (136657 allocations: 7.73 MiB)


552